# Синусоидальное позиционное кодирование

## Описание задачи

Трансформеры изначально не знают порядок токенов в последовательности, так как механизм внимания (self-attention) инвариантен к перестановкам. Поэтому к входным эмбеддингам добавляют позиционные кодировки, чтобы явно передать информацию о позиции токена в последовательности.

В оригинальной архитектуре Transformer используются фиксированные синусоидальные позиционные кодировки — это детерминированные функции от позиции. Они основаны на синусе и косинусе с разными частотами, что позволяет модели учиться обращать внимание на относительные позиции токенов.

## Почему синусоидальное позиционное кодирование важно

Синусоидальные позиционные кодировки важны по нескольким причинам:

- **Нет обучаемых параметров**: это фиксированные функции, их не нужно обучать.
- **Экстраполяция**: такие кодировки можно вычислять для позиций больше, чем встречались в обучении.
- **Относительные позиции**: благодаря тригонометрическим тождествам относительное положение можно выразить через линейные комбинации кодировок.
- **Базовый подход**: это классический базовый вариант позиционного кодирования, от которого отталкиваются многие другие методы.
- **Интерпретируемость**: формулы просты и понятны математически.

## Математическая формулировка

Пусть у нас есть позиция `pos` и размерность `i` (номер компоненты в векторе кодировки).

Для чётных индексов размерности (2i):

`{latex}PE_{(pos, 2i)} = \sin\left(\frac{pos}{10000^{2i/d_{model}}}\right)`

Для нечётных индексов размерности (2i + 1):

`{latex}PE_{(pos, 2i+1)} = \cos\left(\frac{pos}{10000^{2i/d_{model}}}\right)`

Где:

- `pos` — позиция в последовательности (0, 1, 2, …), обычно 0-индексация.
- `i` — индекс размерности (от 0 до `d_model/2 - 1`).
- `d_model` — размерность модельного пространства (размер эмбеддинга).
- `10000^{2i/d_model}` — «длина волны» / частота, зависящая от `i`: для разных размерностей разные частоты.

В итоге матрица позиционных кодировок имеет форму `(max_len, d_model)`, где каждая строка соответствует одной позиции в диапазоне от 0 до `max_len - 1`.

## Интуиция дизайна

Основные свойства такого кодирования:

- **Разные частоты**: каждая размерность использует свою частоту синусоиды.
- **Чётные / нечётные индексы**: на чётных размерностях — синус, на нечётных — косинус.
- **Относительные смещения**: кодировка `PE_{pos+k}` может быть выражена как линейная функция от `PE_{pos}`, что позволяет модели учитывать относительные расстояния между позициями.
- **Разные длины волн**: от `2π` до примерно `2π · 10000`, то есть есть и высокочастотные, и низкочастотные компоненты.

## Что нужно реализовать

Нужно реализовать функцию:

```python
def sinusoidal_positional_encoding(max_len, d_model):
    """
    Generate sinusoidal positional encodings.

    max_len: int, maximum sequence length
    d_model: int, model dimension (must be even)

    returns: np.ndarray of shape (max_len, d_model), dtype=np.float32
    """


In [ ]:
import numpy as np

def sinusoidal_positional_encoding(max_len, d_model):
    if d_model % 2 != 0: raise ValueError("s_model must be even")
    postition = np.arange(max_len, dtype=np.float32)[:, np.newaxis]
    even_dims = np.arange(0, d_model, 2, dtype=np.float32)
    div_term = np.exp(-(np.log(10000.0) * even_dims / d_model))

    angles = postition * div_term
    pe = np.zeros((max_len, d_model), dtype=np.float32)
    pe[:, 0::2] = np.sin(angles)
    pe[:, 1::2] = np.cos(angles)
    return pe